In [2]:
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
data_folder = Path("/Users/tylersojka/Desktop/Bootcamp_Work/Movies-ETL/Data/")

In [4]:
file_to_open = data_folder / "wikipedia-movies.json"

In [5]:
with open(file_to_open, mode='r') as file:
    wiki_movies_raw = json.load(file)


In [6]:
len(wiki_movies_raw)

7311

In [7]:
# first 5 records 
wiki_movies_raw[:5]


[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [11]:
# last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [12]:
# some records in the middle
#wiki_movies_raw[3600:3605]

In [13]:
kaggle_metadata = pd.read_csv(data_folder / "movies_metadata.csv")
ratings = pd.read_csv(data_folder / "ratings.csv")

In [14]:
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [15]:
#sample 5 random entries in ratings
ratings.sample(n=5)

,userId,movieId,rating,timestamp
3433082,35702,6979,3.0,1078008757
11844612,122831,2118,3.5,1228789115
11155158,115556,653,5.0,864912752
8437623,86903,2439,1.0,945902327
7978113,82230,315,1.0,839834910


In [16]:
#sample 5 random entries in kaggle data
kaggle_metadata.sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
36063,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,156220,tt2257602,en,The Staircase II: The Last Chance,Explosive developments - implicating both the ...,...,2012-11-15,0.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Staircase II: The Last Chance,False,7.8,8.0
18406,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,45971,tt0030996,en,You and Me,"Mr. Morris, the owner of a large metropolitan ...",...,1938-06-03,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every time she says 'I Love You'... she breaks...,You and Me,False,8.4,5.0
13442,False,NaN,12000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",NaN,17317,tt0910934,en,Urban Justice,Seagal plays a man with a dark and violent pas...,...,2007-01-01,0.0,96.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Urban Justice,False,5.4,34.0
45001,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 16, 'nam...",NaN,67713,tt0028119,en,Popeye the Sailor Meets Sindbad the Sailor,"In ""Popeye Meets Sindbad (Bluto),"" Popeye's pe...",...,1936-11-27,0.0,16.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Popeye the Sailor Meets Sindbad the Sailor,False,6.4,10.0
27202,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,58526,tt0096160,it,Splendor,Splendor is the name of an old movie theater m...,...,1989-01-01,0.0,0.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Splendor,False,6.8,8.0


In [8]:
# see what happens if you iport the wiki data right into a dataframe
wiki_movies_df = pd.DataFrame(wiki_movies_raw)


In [10]:
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# take a look at the columns in a list
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [13]:
# create a list comprehension to filter only movies with directiors and an imdb link
wiki_movies = [movie for movie in wiki_movies_raw
                if('Director' in movie or 'Directed by' in movie)
                    and 'imdb_link' in movie]
len(wiki_movies)

7080

In [25]:
# put the filtered json into a dataframe, and check how many columns
wiki_movies_df = pd.DataFrame(wiki_movies)
len(wiki_movies_df.columns.tolist())

75

In [27]:
# more list comprehension with an additional filter to filter OUT tvshows(anthing with a season)
wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                    and 'imdb_link' in movie
                    and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [28]:
wiki_movies_df = pd.DataFrame(wiki_movies)
len(wiki_movies_df.columns.tolist())

75

In [29]:
# make a function to make a copy of the movie and return it
def clean_movie(movie):
    movie_copy - dict(movie)
    return movie


In [32]:
wiki_movies_df.sample(n=5)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6383,https://en.wikipedia.org/wiki/The_Age_of_Adaline,2015,https://www.imdb.com/title/tt1655441/,The Age of Adaline,Lee Toland Krieger,"[Sidney Kimmel, Gary Lucchesi, Tom Rosenberg]",NaN,NaN,NaN,"[Blake Lively, Michiel Huisman, Kathy Baker, H...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6618,https://en.wikipedia.org/wiki/Lovesong_(film),2017,https://www.imdb.com/title/tt4305148/,Lovesong,So Yong Kim,"[Alex Lipschultz, Bradley Rust Gray, David Han...",NaN,NaN,NaN,"[Jena Malone, Riley Keough, Brooklyn Decker, A...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5521,https://en.wikipedia.org/wiki/Christmas_with_t...,2012,https://www.imdb.com/title/tt1954334/,Christmas with the Dead,Terrill Lee Lankford,"[William Arscott, Brad Maule, Marion Arscott]",NaN,NaN,"["", Christmas with the Dead, "", by Joe R. Lans...","[Damian Maffei, Brad Maule, Kasey Lansdale, Ch...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6695,https://en.wikipedia.org/wiki/Transformers:_Th...,2017,https://www.imdb.com/title/tt3371366/,Transformers: The Last Knight,Michael Bay,"[Lorenzo di Bonaventura, Tom DeSanto, Don Murp...","[Art Marcum, Matt Holloway, Ken Nolan]","[Akiva Goldsman, Art Marcum, Matt Holloway, Ke...","[Transformers, by, Hasbro]","[Mark Wahlberg, Josh Duhamel, Stanley Tucci, A...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2640,https://en.wikipedia.org/wiki/Gladiator_(2000_...,2000,https://www.imdb.com/title/tt0172495/,Gladiator,Ridley Scott,"[Douglas Wick, David Franzoni, Branko Lustig]","[David Franzoni, John Logan, William Nicholson]",David Franzoni,NaN,"[Russell Crowe, Joaquin Phoenix, Connie Nielse...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
wiki_movies_df[wiki_movies_df['Simplified'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
1534,"https://en.wikipedia.org/wiki/East_Palace,_Wes...",1996,https://www.imdb.com/title/tt0119007/,"East Palace, West Palace",Zhang Yuan,"[Christophe Jung, Christophe Ménager, Zhang Yu...",NaN,NaN,NaN,"[Si Han, Hu Jun, Zhao Wei]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6456,https://en.wikipedia.org/wiki/Ip_Man_3,2016,https://www.imdb.com/title/tt2888046/,Ip Man 3,Wilson Yip,Raymond Wong,NaN,NaN,NaN,"[Donnie Yen, Zhang Jin, Lynn Hung, Patrick Tam...",...,NaN,NaN,"[Jip, 6, Man, 6, Saam, 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6613,https://en.wikipedia.org/wiki/The_Great_Wall_(...,2017,https://www.imdb.com/title/tt2034800/,The Great Wall,Zhang Yimou,"[Thomas Tull, Charles Roven, Jon Jashni, Peter...","[Carlo Bernard, Doug Miro, Tony Gilroy]","[Max Brooks, Edward Zwick, Marshall Herskovitz]",NaN,"[Matt Damon, Jing Tian, Pedro Pascal, Willem D...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6661,https://en.wikipedia.org/wiki/Born_in_China,2017,https://www.imdb.com/title/tt4629266/,Born in China,Lu Chuan,"[Roy Conli, Brian Leith, Phil Chapman]","[David Fowler, Brian Leith, Phil Chapman, Lu C...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
wiki_movies_df[wiki_movies_df['Romanized'].notnull()]['url']

6834    https://en.wikipedia.org/wiki/The_Insult_(film)
6849      https://en.wikipedia.org/wiki/Loveless_(film)
Name: url, dtype: object

In [35]:
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplifie

In [49]:
# complete list of columns that hold alternate title data: Also known as, Arabic, Cantonese, Chinese, French, Hangul, Hebrew, Hepburn, Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  Revised Romanization, Romanized, Russian, Simplified, Traditional, Yiddish

In [52]:
def clean_movie(movie):
    movie = dict(movie) #create a non destructive copy   
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',  # loop through a list of all alt title keys
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:    ## Check if the current key exists in the movie object.
            alt_titles[key] = movie[key]       # If so, remove the key-value pair and
            movie.pop(key)                      # add to the alternative titles dictionary.
    if len(alt_titles) > 0:                   # After looping through every key, add the
        movie['alt_titles'] = alt_titles        # alternative titles dict to the movie object.

    return movie

In [53]:
# make a list of cleaned movies with a list comprehension:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [54]:
# Set wiki_movies_df to be the DataFrame created from clean_movies, and print out a list of the columns.
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']